## LB Sco	0.64727



In [1]:
## load data
library(data.table)
train <- fread("train_data.csv")
test <- fread("test_data.csv")

In [2]:

## check missing values - no missing values
colSums(is.na(train))
colSums(is.na(test))

## check target class
train[,.N/nrow(train), target]


connection_id        cont_1        cont_2        cont_3        cont_4 
            0             0             0             0             0 
       cont_5        cont_6        cont_7        cont_8        cont_9 
            0             0             0             0             0 
      cont_10       cont_11       cont_12       cont_13       cont_14 
            0             0             0             0             0 
      cont_15       cont_16       cont_17       cont_18         cat_1 
            0             0             0             0             0 
        cat_2         cat_3         cat_4         cat_5         cat_6 
            0             0             0             0             0 
        cat_7         cat_8         cat_9        cat_10        cat_11 
            0             0             0             0             0 
       cat_12        cat_13        cat_14        cat_15        cat_16 
            0             0             0             0             0 
       cat_17        cat_18        cat_19        cat_20        cat_21 
            0             0             0             0             0 
       cat_22        cat_23        target 
            0             0             0

connection_id        cont_1        cont_2        cont_3        cont_4 
            0             0             0             0             0 
       cont_5        cont_6        cont_7        cont_8        cont_9 
            0             0             0             0             0 
      cont_10       cont_11       cont_12       cont_13       cont_14 
            0             0             0             0             0 
      cont_15       cont_16       cont_17       cont_18         cat_1 
            0             0             0             0             0 
        cat_2         cat_3         cat_4         cat_5         cat_6 
            0             0             0             0             0 
        cat_7         cat_8         cat_9        cat_10        cat_11 
            0             0             0             0             0 
       cat_12        cat_13        cat_14        cat_15        cat_16 
            0             0             0             0             0 
       cat_17        cat_18        cat_19        cat_20        cat_21 
            0             0             0             0             0 
       cat_22        cat_23 
            0             0

target,V1
2,0.2176756
0,0.5839570
1,0.1983675


In [3]:
# Model 1 -----------------------------------------------------------------

## lets predict the majority class
samplesub <- fread("sample_submission.csv")
samplesub[, target := 0]
fwrite(samplesub, "sub0.csv")


connection_id,target
cxcon_2,0
cxcon_5,0
cxcon_8,0
cxcon_11,0
cxcon_14,0
cxcon_17,0
cxcon_20,0
cxcon_23,0
cxcon_26,0
cxcon_29,0


In [14]:

# Model 2 + 3 -----------------------------------------------------------------
## Random Forest
## Gradient Boosting

library(xgboost)
library(caTools)

getXGBData <- function(train, test)
{
  split <- sample.split(Y = train$target, SplitRatio = 0.5)
  
  dtrain <- train[split]
  dvalid <- train[!split]
  
  dtrain <- xgb.DMatrix(data = as.matrix(dtrain[,-c('connection_id','target'), with=F]), label = dtrain$target)
  dvalid <- xgb.DMatrix(data = as.matrix(dvalid[, -c('connection_id', 'target'), with = F]), label = dvalid$target)
  dtest <- xgb.DMatrix(data = as.matrix(test[,-c('connection_id'), with=F]))
  
  return (list(train = dtrain, test = dtest, eval = dvalid))
  
}

getMulAcc <- function(pred, dtrain)
{
  label <- getinfo(dtrain, "label")
  acc <- mean(label == pred)
  return(list(metric = "maccuracy", value = acc))
}


runModels <- function(dtrain, dtest, dvalid, XGB = 0) ## set 1 for XGB: Default run is RF
{
  
  if(XGB == 1)
  {
    cat('Running Gradient Boosting Model...\n\n')
    # default parameters
    params <- list(objective = 'multi:softmax',num_class =4 )
    
    watchlist <- list('train' = dtrain, 'valid' = dvalid)
    clf <- xgb.train(params
                     ,dtrain
                     ,100
                     ,watchlist
                     ,feval = getMulAcc
                     ,print_every_n = 20
                     ,early_stopping_rounds = 20
                     ,maximize = T
    )
    
    
    pred <- predict(clf, dtest)
    
  } else if (XGB == 0) {
    
    cat('Running Random Forest Model...\n\n')
    params <- list(
                   objective = 'multi:softmax'
                   ,num_class = 3
                   ,normalize_type = 'tree'
                
                   )
    
    watchlist <- list('train' = dtrain, 'valid' = dvalid)
    clf <- xgb.train(params
                     ,dtrain
                     ,100
                     ,watchlist
                     ,feval = getMulAcc
                     ,print_every_n = 20
                     ,early_stopping_rounds =30
                     ,maximize = T
                
    )
    
    
    pred <- predict(clf, dtest)
    
  } 
  
  return(pred)

}

xgbdata <- getXGBData(train, test)

predsRF <- runModels(dtrain = xgbdata$train, dtest = xgbdata$test, dvalid = xgbdata$eval)
predsXGB <- runModels(dtrain = xgbdata$train, dtest = xgbdata$test, dvalid = xgbdata$eval,XGB = 1)


## make submissions
sampsub1 <- fread("sample_submission.csv")
sampsub1[, target := predsRF]
fwrite(sampsub1, "sub_1.csv")

sampsub2 <- fread("sample_submission.csv")
sampsub2[, target := predsXGB]
fwrite(sampsub2, "sub_2.csv")


Running Random Forest Model...

[1]	train-maccuracy:0.779086	valid-maccuracy:0.778475 
Multiple eval metrics are present. Will use valid_maccuracy for early stopping.
Will train until valid_maccuracy hasn't improved in 30 rounds.

[21]	train-maccuracy:0.782240	valid-maccuracy:0.780530 
[41]	train-maccuracy:0.783268	valid-maccuracy:0.780223 
Stopping. Best iteration:
[17]	train-maccuracy:0.782063	valid-maccuracy:0.780554

Running Gradient Boosting Model...

[1]	train-maccuracy:0.779075	valid-maccuracy:0.778427 
Multiple eval metrics are present. Will use valid_maccuracy for early stopping.
Will train until valid_maccuracy hasn't improved in 20 rounds.

[21]	train-maccuracy:0.782158	valid-maccuracy:0.780566 
Stopping. Best iteration:
[15]	train-maccuracy:0.781839	valid-maccuracy:0.780589



connection_id,target
cxcon_2,1
cxcon_5,0
cxcon_8,0
cxcon_11,0
cxcon_14,0
cxcon_17,0
cxcon_20,0
cxcon_23,1
cxcon_26,0
cxcon_29,0


connection_id,target
cxcon_2,1
cxcon_5,0
cxcon_8,0
cxcon_11,0
cxcon_14,0
cxcon_17,0
cxcon_20,0
cxcon_23,1
cxcon_26,0
cxcon_29,0


In [10]:
#0.782051 boost,0.782063 rf,3,40,5,20 score:0.78128, 5,20,3,40 score:0.78117